### Drive Mount

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


### Data Load

In [2]:
import glob
import os

img_lst = glob.glob("/content/drive/MyDrive/Colab_Notebooks/안벽/Prototype_v3/train/images/"+"*.jpg")+glob.glob("/content/drive/MyDrive/Colab_Notebooks/안벽/Prototype_v3/train/images/"+"*.JPG")+glob.glob("/content/drive/MyDrive/Colab_Notebooks/안벽/Prototype_v3/train/images/"+"*.jpeg")
bbox_lst = glob.glob("/content/drive/MyDrive/Colab_Notebooks/안벽/Prototype_v3/train/labels/"+"*.txt")

img_lst.sort()
bbox_lst.sort()

len(img_lst), len(bbox_lst)

(791, 791)

### Function Define

cv2.equalize()는 그레이스케일 영상만 지원합니다.

따라서 BGR을 YCrCb로 전환 split()으로 나눠준 뒤에 Y차원 요소들만<br>
평활화 작업을 거친 후 merge로 결합하고 다시 BGR 영상으로 전환해야 합니다.

In [52]:
import cv2
import numpy as np
import random
random.seed(1027)

def image_equalizeHist(img_item):
  im = cv2.imread(img_item)
  im_ycrcb = cv2.cvtColor(im, cv2.COLOR_BGR2YCrCb ) 
  ycrcb_planes = cv2.split(im_ycrcb)
  # 튜플이라서 형변환
  ycrcb_planes = list(ycrcb_planes)
  # 밝기 성분에 대해서만 히스토그램 평활화 수행
  temp = cv2.equalizeHist(ycrcb_planes[0]) 
  ycrcb_planes[0] = temp
  # 원상복귀
  ycrcb_planes = tuple(ycrcb_planes)

  dst_ycrcb = cv2.merge(ycrcb_planes)
  transformed_image = cv2.cvtColor(dst_ycrcb, cv2.COLOR_YCrCb2BGR)
  
  return transformed_image

### Image Data Preprocessing & Save

In [4]:
import os

def createDirectory(directory):
    try:
        if not os.path.exists(directory):
            os.makedirs(directory)
    except OSError:
        print("Error: Failed to create the directory.")

In [5]:
from tqdm import tqdm

sample_preds_folder = "/content/drive/MyDrive/Colab_Notebooks/안벽/Prototype_v5/"
dir_img = "hist_img/"
dir_bbox = "hist_bbox/"

In [6]:
# Create Directory

createDirectory(sample_preds_folder+dir_img)
createDirectory(sample_preds_folder+dir_bbox)

In [53]:
import shutil

# 전처리
for img_item, bbox_item in tqdm(zip(img_lst,bbox_lst)):

    transformed_image = image_equalizeHist(img_item) 
    cv2.imwrite(os.path.join(sample_preds_folder,dir_img,"{}{}.jpg".format(img_item[72:-4],"_eh")),transformed_image) # 새로운 이미지 저장     

    f_name = os.path.join(sample_preds_folder,dir_bbox,"{}{}.txt".format(bbox_item[72:-4],"_eh"))
    shutil.copy(bbox_item, f_name)

791it [08:57,  1.47it/s]


----------------------------------------------------------------